In [1]:
import cupy as np
#import numpy as np
import numpy as np_cpu
import time
import os
import csv
import copy    # chap 9
import wave    # chap 11
import matplotlib.pyplot as plt

from PIL import Image
from IPython.core.display import HTML # chap 14

In [2]:
def relu(x):
    return np.maximum(x, 0)

def relu_derv(y):
    return np.sign(y)

In [ ]:
def leaky_relu(x):
    return np.maximum(x, 0.01*x)

def leaky_relu_derv(y):
    return np.where(y<0,0.01,1)

In [3]:
def sigmoid(x):
    return 1 / (1.0 + np.exp(-x))
        
def sigmoid_derv(y):
    return y * (1 - y)

def sigmoid_cross_entropy_with_logits(z, x):
    return x - x * z + np.log(1 + np.exp(-x))

def sigmoid_cross_entropy_with_logits_derv(z, x):
    return -z + sigmoid(x)

In [4]:
def tanh(x):
    return 2 * sigmoid(2*x) - 1

def tanh_derv(y):
    return (1.0 + y) * (1.0 - y)

In [5]:
def softmax(x):
    
    max_elem = np.max(x, axis=1)
    
    diff = (x.transpose()-max_elem).transpose()
    
    exp = np.exp(diff)
    sum_exp=np.sum(exp, axis=1)
    probs = (exp.transpose() / sum_exp).transpose()  #b,a / a .trns
    #print("probs",probs)
    #print("x",x)
    """
    for i in range(probs.shape[0]):
        for j in range(probs.shape[1]):
            if probs[i,j]<1e-9:
                probs[i,j]=1e-9
    #print("after",probs)
       """     
    return probs
"""
def softmax_(x):
    x=x-np.max(x)
    exp_=np.exp(x)
    sum_=np.sum(x,axis=1)
    return exp_/sum_

def softmax_derv(x):
    derv_ = softmax_(x)*(1-softmax_(x))
    sum_=np.sum(x,axis=1)
    exp_=np.exp(x)
    return exp_/sum_
"""

def softmax_derv(x):
    if len(x.shape)!=2:
        raise Exception("소프트맥스역전파에러")
    #print(x.shape)
    temp_batch_size=x.shape[0]
    temp_output_size=x.shape[1]
    x_temp=np.zeros((temp_batch_size,temp_output_size,temp_output_size), dtype='float32')
    x_temp_after=np.zeros((temp_batch_size,temp_output_size,temp_output_size), dtype='float32')
    for i in range(temp_batch_size):
        x_temp[i,:,:]=np.diag(x[i,:])
        #print(x[i,:].reshape(x.shape[1],1).shape)
        temp=np.tile(x[i,:].reshape(x.shape[1],1),x.shape[1])
        #print(temp.shape)
        x_temp_after[i,:,:]=temp*np.transpose(temp)
        x_temp[i,:,:] = x_temp[i,:,:]-x_temp_after[i,:,:]
        
    
    #https://aimatters.wordpress.com/2019/06/17/the-softmax-function-derivative/
    return np.sum(x_temp,axis=1)
"""
def softmax_derv(x,y):
    mb_size, nom_size = x.shape
    derv = np.ndarray([mb_size,nom_size,nom_size])
    for n in range(mb_size):
        for i in range(nom_size):
            for j in range(nom_size):
                derv[n,i,j]=-y[n,i]*y[n,j]
            derv[n,i,i]+=y[n,i]
    return derv
    """
def softmax_cross_entropy_with_logits(labels, logits):
    probs = softmax(logits)
    #print(labels.shape)
    #print(logits.shape,"d")
    #print("logis",logits)
    #print("prob",probs)
    
    return -np.sum(labels * np.log(probs+1.0e-10), axis=1)

def softmax_cross_entropy_with_logits_derv(labels, logits):
    return softmax(logits) - labels

In [6]:
def load_csv(path, skip_header=True):
    with open(path) as csvfile:
        csvreader = csv.reader(csvfile)
        headers = None
        if skip_header: headers = next(csvreader, None)
        rows = []
        for row in csvreader:
            rows.append(row)
            
    return rows, headers

In [7]:
def onehot(xs, cnt):
    return np.eye(cnt)[np.array(xs).astype(int)]

def vector_to_str(x, fmt='%.2f', max_cnt=0):
    if max_cnt == 0 or len(x) <= max_cnt:
        return '[' + ','.join([fmt]*len(x)) % tuple(x) + ']'
    v = x[0:max_cnt]
    return '[' + ','.join([fmt]*len(v)) % tuple(v) + ',...]'

In [8]:
def load_image_pixels(imagepath, resolution, input_shape):
    img = Image.open(imagepath)
    resized = img.resize(resolution)
    return np.array(resized).reshape(input_shape)

def draw_images_horz(xs, image_shape=None):
    show_cnt = len(xs)
    fig, axes = plt.subplots(1, show_cnt, figsize=(5,5))
    for n in range(show_cnt):
        img = xs[n]
        if image_shape:
            x3d = img.reshape(image_shape)
            img = Image.fromarray(np.uint8(x3d))
        axes[n].imshow(img)
        axes[n].axis('off')
    plt.draw()
    plt.show()

In [9]:
def show_select_results(est, ans, target_names, max_cnt=0):
    for n in range(len(est)):
        pstr = vector_to_str(100*est[n], '%2.0f', max_cnt)
        estr = target_names[np.argmax(est[n])]
        astr = target_names[np.argmax(ans[n])]
        rstr = 'O'
        if estr != astr: rstr = 'X'
        print('추정확률분포 {} => 추정 {} : 정답 {} => {}'. \
              format(pstr, estr, astr, rstr))

In [ ]:
def list_dir(path):
    filenames = os.listdir(path)
    filenames.sort()
    return filenames